In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from datetime import datetime
import calendar

In [3]:
df_v1=pd.read_csv('merged_movies_credits.csv')

In [4]:
df_v1.columns

Index(['budget', 'genres', 'id', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_day', 'cast',
       'director_name', 'crew'],
      dtype='object')

In [5]:
df_v1.shape

(45386, 20)

In [6]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45386 entries, 0 to 45385
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45386 non-null  float64
 1   genres                45386 non-null  object 
 2   id                    45386 non-null  int64  
 3   original_language     45386 non-null  object 
 4   popularity            45386 non-null  float64
 5   production_companies  45386 non-null  object 
 6   production_countries  45386 non-null  object 
 7   release_date          45386 non-null  object 
 8   revenue               45386 non-null  float64
 9   runtime               45386 non-null  float64
 10  spoken_languages      45386 non-null  object 
 11  title                 45386 non-null  object 
 12  vote_average          45386 non-null  float64
 13  vote_count            45386 non-null  int64  
 14  return                45386 non-null  float64
 15  release_year       

In [7]:
df_v1.to_json('merged_movies_credits.json')

### Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

In [8]:
# Cambiamos las fechas de la columna 'release_date' a formato AAAA-mm-dd, ignorando con 'coerce'
# los valores que no cumplen con el criterio para la funcion .to_datetime
df_v1["release_date"] = pd.to_datetime(df_v1['release_date'],errors='coerce')


In [9]:
# Funcion 1: def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. 
# Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
#  Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

from dateutil.parser import parse

def cantidad_filmaciones_mes(mes):
# Mapear los nombres de los meses en español a los números de mes correspondientes
    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }
# Obtener el número del mes a partir del nombre en español
    mes_numero = meses.get(mes.lower())

    if mes_numero is None:
        raise ValueError('Mes inválido: {}'.format(mes))

    # Filtrar las películas por mes y contar la cantidad
    cantidad = df_v1[df_v1['release_date'].dt.month == mes_numero].shape[0]

    return f"{cantidad} es la cantidad de películas que fueron estrenadas en el mes de {mes}."

In [10]:
cantidad_filmaciones_mes('febrero')

'3034 es la cantidad de películas que fueron estrenadas en el mes de febrero.'

In [11]:
# Funcion 2: def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. 
# Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
# Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

def cantidad_filmaciones_dia(dia):
    # Mapear los nombres de los días en español a los números de día correspondientes
    dias = {
        'lunes': 0,
        'martes': 1,
        'miércoles': 2,
        'jueves': 3,
        'viernes': 4,
        'sábado': 5,
        'domingo': 6
    }

# Obtener el número del día a partir del nombre en español
    dia_numero = dias.get(dia.lower())

    if dia_numero is None:
        raise ValueError('Día inválido: {}'.format(dia))

# Filtrar las películas por día de la semana y contar la cantidad
    cantidad = df_v1[df_v1['release_date'].dt.dayofweek == dia_numero].shape[0]

    return f"{cantidad} es la cantidad de películas que fueron estrenadas en los dias {dia}."


In [12]:
cantidad_filmaciones_dia('martes')

'4641 es la cantidad de películas que fueron estrenadas en los dias martes.'

In [13]:
# Funcion 3 : def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como 
# respuesta el título, el año de estreno y el score.
# Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

def score_titulo_2(titulo_de_la_filmación):
    # Realizamos una búsqueda case insensitive y con coincidencia parcial
    coincidencia = df_v1['title'].str.contains(titulo_de_la_filmación, case=False, na=False)

    # Filtramos el DataFrame para obtener las filas con coincidencias
    # df_v1[coincidencia]

    # Si hay coincidencias, seleccionamos la primera fila y obtenemos los valores
    if not df_v1[coincidencia].empty:
        row = df_v1[coincidencia].iloc[0]
        return f"La pelicula {row['title']} fue estrenada en el año {row['release_year']} con un score de {row['vote_average']}"

    return None

In [14]:
score_titulo_2('toy st')

'La pelicula Toy Story fue estrenada en el año 1995 con un score de 7.7'

In [15]:
# Funcion 4: def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el 
# título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 
# 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, 
# no se devuelve ningun valor.
# Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, 
# con un promedio de X.

def votos_titulo(titulo_de_la_filmación):
    # Realizamos una búsqueda case insensitive y con coincidencia parcial
    matches = df_v1['title'].str.contains(titulo_de_la_filmación, case=False, na=False)

    # Filtramos el DataFrame para obtener las filas con coincidencias
    df_matches = df_v1[matches]

    # Si hay coincidencias, seleccionamos la primera fila y verificamos el número de valoraciones
    if not df_matches.empty:
        row = df_matches.iloc[0]
        vote_count = row['vote_count']
        if vote_count < 2000:
            return f"La pelicula {row['title']} no obtuvo suficientes valoraciones"
        return f"La pelicula {row['title']} fue estrenada en el año {row['release_year']} con un total de {row['vote_count']} valoraciones y un promedio de {row['vote_average']}"

    


In [16]:
votos_titulo('toy S')

'La pelicula Toy Story fue estrenada en el año 1995 con un total de 5415 valoraciones y un promedio de 7.7'

In [17]:
votos_titulo('Grumpier Old Men')

'La pelicula Grumpier Old Men no obtuvo suficientes valoraciones'

In [18]:
# Funcion 5: def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset 
# debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que 
# ha participado y el promedio de retorno. La definición no deberá considerar directores.
# Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X 
# con un promedio de X por filmación.

def get_actor(nombre_actor):
    # Filtrar las filas del DataFrame donde el nombre del actor coincide
    actor_films = df_v1[df_v1['cast'].str.contains(nombre_actor, case=False, na=False)]
    
    # Calcular la cantidad de películas en las que ha participado el actor
    cantidad_peliculas = len(actor_films)
    
    # Calcular el promedio de la columna 'return' para el actor
    promedio_return = actor_films['return'].mean()
    
    # Calcular el éxito del actor mediante la columna 'return'
    exito_actor = actor_films['return'].sum()
    
    return exito_actor, cantidad_peliculas, promedio_return

In [19]:
get_actor('Brad p')

(inf, 66, inf)

In [20]:
def get_actor_2(nombre_actor):
    actor_films = df_v1[df_v1['cast'].str.contains(nombre_actor, case=False, na=False)]
    cantidad_peliculas = len(actor_films)
    
    # Inicializar el éxito del actor y el total de 'return'
    exito_actor = 0
    total_return = 0
    
    # Calcular el éxito del actor y el total de 'return'
    for _, row in actor_films.iterrows():
        budget = row['budget']
        revenue = row['revenue']
        if revenue != 0:
            return_value = budget / revenue
            exito_actor += return_value
            total_return += 1
    
    # Calcular el promedio de 'return'
    promedio_return = exito_actor / total_return if total_return != 0 else 0
    
    #return round(exito_actor,3), cantidad_peliculas, round((promedio_return*100),2)
    return f"""El actor {nombre_actor} ha participado de {cantidad_peliculas} filmaciones.
    El mismo ha conseguido un retorno de {round(exito_actor,3)}M con un promedio de {round((promedio_return*100),2)}M por filmación."""

In [21]:
get_actor_2('brad pi')

'El actor brad pi ha participado de 60 filmaciones.\n    El mismo ha conseguido un retorno de 29.641M con un promedio de 63.07M por filmación.'

In [60]:
# Funcion 6: def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro 
# de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver:
# => el nombre de cada película con la fecha de lanzamiento, 
# => retorno individual, costo y ganancia de la misma.

def get_director_2(nombre_director):
    director_films = df_v1[df_v1['director_name'].str.contains(nombre_director, case=False, na=False)]
        
    # Inicializar el éxito del actor y el total de 'return'
    exito_director = 0
    total_return = 0
    
    # Calcular el éxito del actor y el total de 'return'
    for _, row in director_films.iterrows():
        budget = row['budget']
        revenue = row['revenue']
        if revenue != 0:
            return_value = budget / revenue
            exito_director += return_value
            total_return += 1
    
    # Calcular el promedio de 'return'
    promedio_return = exito_director / total_return if total_return != 0 else 0
    
    #return round(exito_actor,3), cantidad_peliculas, round((promedio_return*100),2)
    return f"""El director {nombre_director} ha conseguido un retorno de {round(exito_director,3)}M con un promedio de {round((promedio_return*100),2)}% por filmación."""

In [61]:
get_director_2('george lucas')

'El director george lucas ha conseguido un retorno de 0.781M con un promedio de 13.01% por filmación.'

In [24]:
df_v1.columns


Index(['budget', 'genres', 'id', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_day', 'cast',
       'director_name', 'crew'],
      dtype='object')

In [72]:
# def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro 
# de un dataset debiendo devolver:
# => el éxito del mismo medido a través del retorno. 
# => el nombre de cada película con la fecha de lanzamiento, 
# => retorno individual, costo y ganancia de la misma.
def get_director(nombre_director):
    peliculas = []
    director_films = df_v1[df_v1['director_name'].str.contains(nombre_director, case=False, na=False)]
    for pelicula in director_films:
        if df_v1['director_name'][1] == nombre_director:
            peliculas.append({
                'nombre': pelicula['title'],
                'fecha_lanzamiento': pelicula['release_date'],
                'retorno': pelicula['return'],
                'costo': pelicula['budget'],
                'ganancia': pelicula['revenue']
            })
    
    if len(peliculas) == 0:
        return "No se encontraron películas para el director especificado."
    
    exito_total = sum(pelicula['return'] for pelicula in peliculas)
    
    return {
        'exito_total': exito_total,
        'peliculas': peliculas
    }


In [73]:
get_director('Tom Hanks')

'No se encontraron películas para el director especificado.'

In [27]:
# def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro 
# de un dataset debiendo devolver:
# => el éxito del mismo medido a través del retorno. 
# => el nombre de cada película con la fecha de lanzamiento, 
# => retorno individual, costo y ganancia de la misma.

def get_director_3(nombre_director):
    # => el nombre de cada película con la fecha de lanzamiento
    # => para busqueda exacta >> peliculas = df[df['director_name'] == nombre_director][['title', 'release_year']]
    peliculas = df_v1[df_v1['director_name'].str.contains(nombre_director,case=False,na=False)][['title', 'release_date','budget','return']]
    
    # => el retorno individual, el éxito del mismo medido a través del retorno.
    exito_director = peliculas['return'].sum()
    costo = peliculas['budget']
    return {'films':peliculas, 'exito': exito_director, 'costo': costo}



In [28]:
get_director_3('george l')

{'films':                                               title release_date       budget   
 256                                       Star Wars   1977-05-25   11000000.0  \
 2514      Star Wars: Episode I - The Phantom Menace   1999-05-19  115000000.0   
 3246                              American Graffiti   1973-08-01     777000.0   
 5246   Star Wars: Episode II - Attack of the Clones   2002-05-15  120000000.0   
 6500                                       THX 1138   1971-03-11     777000.0   
 10074  Star Wars: Episode III - Revenge of the Sith   2005-05-17  113000000.0   
 14112                                    A Dog Year   2009-09-03          0.0   
 24621             Electronic Labyrinth THX 1138 4EB   1967-03-14          0.0   
 30601                              Traffic in Souls   1913-11-24          0.0   
 32624                         Vanishing of the Bees   2009-10-09          0.0   
 32874                       The Conquest of Everest   1953-12-09          0.0   
 39758 

In [ ]:
# Funcion ML => def recomendacion( `titulo` ):
#    Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

